In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from pydicom import dcmread
import numpy as np
import pandas as pd
import cv2
from skimage.util import montage
from skimage import io
import shutil

%matplotlib inline
tf.__version__

import sys
print(sys.version)
print(sys.executable)

import math
from PIL import Image, ImageDraw, ImageColor
import json
import glob
import os
import cv2
import nibabel as nib

3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]
c:\ProgramData\Anaconda3\python.exe


In [22]:
# draw dicom images with metadata info
from datetime import datetime
def drawCategorizedImages(folderPath, dict, fontsize=10, fontfamily='MingLiU'):
    for patient in dict:
        for study in dict[patient]:
            for series in dict[patient][study]:
                fileList = dict[patient][study][series]
                imagesList = []
                for file in fileList:
                    imagesList.append(dcmread(folderPath + file['dcmPath']))
                imagesList.sort(key=lambda x: x.InstanceNumber, reverse=False) # sort image by instance number
                quantity = len(imagesList)
                cols = 10
                rows = math.ceil(quantity/cols)
                rows = rows + int(rows/6) + 1 # add blank rows to place text
                fig = plt.figure(figsize=(cols*2,rows*2))
                flag = True
                for i in range(quantity):
                    ds = imagesList[i]

                    if ("PixelData" in ds):
                        #add text only once
                        if flag:
                            plt.text(0.01,0.14,"PatientName: " + str(ds.PatientName), fontsize=fontsize, fontfamily=fontfamily)
                            plt.text(0.01,0.11,"PatientID: " + str(ds.PatientID), fontsize=fontsize, fontfamily=fontfamily)
                            plt.text(0.01,0.08,"Sex: " + str(ds.PatientSex), fontsize=fontsize, fontfamily=fontfamily)
                            plt.text(0.33,0.14,"InstitutionName: " + ds.InstitutionName if ds.InstitutionName != "" else "InstitutionName: None", fontsize=fontsize, fontfamily=fontfamily)
                            plt.text(0.33,0.11,"StudyDescription: " + ds.StudyDescription, fontsize=fontsize, fontfamily=fontfamily)
                            plt.text(0.33,0.08,"StudyID: " + ds.StudyID, fontsize=fontsize, fontfamily=fontfamily)
                            plt.text(0.66,0.14,"SeriesNumber: " + str(ds.SeriesNumber), fontsize=fontsize, fontfamily=fontfamily)
                            plt.text(0.66,0.11,"SeriesDescription: " + str(ds.SeriesDescription), fontsize=fontsize, fontfamily=fontfamily)
                            plt.text(0.66,0.08,"AccessionNumber: " + ds.AccessionNumber, fontsize=fontsize, fontfamily=fontfamily)
                            plt.text(0.66,0.05,"AcquisitionDate: " + str(datetime.strptime(ds.AcquisitionDate, '%Y%m%d').date()), fontsize=fontsize, fontfamily=fontfamily)
                            plt.text(0.66,0.02,"AcquisitionTime: " + ds.AcquisitionTime, fontsize=fontsize, fontfamily=fontfamily)
#                             plt.text(0.66,0.085,"SliceThickness: " + str(math.ceil(ds.SliceThickness)) + "mm", fontsize=fontsize, fontfamily=fontfamily)
#                             plt.text(0.66,0.06,"SliceLocation: " + str(math.ceil(ds.SliceLocation)) + "mm", fontsize=fontsize, fontfamily=fontfamily)
                            flag = False   

                        fig.add_subplot(rows, cols, i+1)
                        if(len(ds.pixel_array.shape)==3):
                            plt.imshow(ds.pixel_array[:,:,1])
                        else:
                            plt.imshow(ds.pixel_array)
                        plt.axis('off')

##################################################################

def showImagesWithPyPlot(images,cmap):
    quantity = len(images)
    cols = 9
    rows = math.ceil(quantity/cols)
    fig,ax = plt.subplots(rows,cols,figsize=(22,2.5*rows))
    for i in range(quantity):
        ax[i//cols][i%cols].imshow(images[i],cmap=cmap)
        ax[i//cols][i%cols].axis('off')

def showNiftiImagesWithPyPlot(nifti,cmap):
    quantity = nifti.shape[2]
    cols = 9
    rows = math.ceil(quantity/cols)
    fig,ax = plt.subplots(rows,cols,figsize=(22,2.5*rows))
    for i in range(quantity):
        ax[i//cols][i%cols].imshow(nifti[:,:,i],cmap=cmap)
        ax[i//cols][i%cols].axis('off')

##################################################################

#使用json檔案畫實心標記區域
def fillMissingPoint(segments):
    startX = segments[0]['a'][0]
    startY = segments[0]['a'][1]
    endX = segments[len(segments)-1]['b'][0]
    endY = segments[len(segments)-1]['b'][1]
    width = segments[0]['width']
    distanceX = abs(endX-startX)
    distanceY = abs(endY-startY)    
    stepX = 1
    if(distanceX<=distanceY):
        stepX = distanceX/distanceY
    stepY = 1
    if(distanceY<=distanceX):
        stepY = distanceY/distanceX
    if(endX >= startX and endY >= startY):
        while(endX >= startX and endY >= startY):
            a = [endX,endY]
            endX-=stepX
            endY-=stepY
            b = [endX,endY]
            segments.append({'a':a,'b':b,'width':width})
    elif(endX >= startX and endY <= startY):
        while(endX >= startX and endY <= startY):
            a = [endX,endY]
            endX-=stepX
            endY+=stepY
            b = [endX,endY]
            segments.append({'a':a,'b':b,'width':width})
    elif(endX <= startX and endY <= startY):
        while(endX <= startX and endY <= startY):
            a = [endX,endY]
            endX+=stepX
            endY+=stepY
            b = [endX,endY]
            segments.append({'a':a,'b':b,'width':width})
    elif(endX <= startX and endY >= startY):
        while(endX <= startX and endY >= startY):
            a = [endX,endY]
            endX+=stepX
            endY-=stepY
            b = [endX,endY]
            segments.append({'a':a,'b':b,'width':width})
    return segments

def getLabeledImageWithJson(jsonPath):
    file = open(jsonPath)
    data = json.load(file)
    image = Image.new("RGB", data['size'])
    draw = ImageDraw.Draw(image)
    shapes = data['shapes']
    for shape in shapes:
        segments = shape['segments']
        segments = fillMissingPoint(segments)
        strokeColor = shape['strokeColor']
        for segment in segments:
            startEndCoord = [(segment['a'][0], segment['a'][1]),(segment['b'][0], segment['b'][1])]
            draw.line(startEndCoord, fill=strokeColor)
    
    # 找輪廓並填充顔色
    grayImage  = cv2.cvtColor(np.asarray(image), cv2.COLOR_BGR2GRAY)
    cnts = cv2.findContours(grayImage, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
        cv2.drawContours(grayImage,[c], 0, (255,255,255), -1)

    colorImage = cv2.cvtColor(grayImage,cv2.COLOR_GRAY2RGB)
    colorImage[np.all(colorImage == (255, 255, 255), axis=-1)] = ImageColor.getcolor(strokeColor, "RGB")    
    
    #convert black background to transparent
    _, alpha = cv2.threshold(grayImage, 0, 255, cv2.THRESH_BINARY)
    b, g, r = cv2.split(colorImage)
    rgba = [b, g, r, alpha]
    colorImage = cv2.merge(rgba, 4)
    
    #convert cv2 to pil format
    pilImage = Image.fromarray(colorImage)
    return pilImage

##################################################################

# categorize dicom files with patientID, studyID and seriesID without using dicomdir
# 查找自己的json標記檔是否存在，如果不存在label設爲no，否則label設爲json標記檔的路徑
def getLabelPath(dcmPath, sopInstanceUID):
    jsonFiles = glob.glob(dcmPath.split('\\')[0] + "/*.json")
    for jsonFile in jsonFiles:
        if(sopInstanceUID in jsonFile):
            return jsonFile
    return 'NA'

# dict: {patientID->studyID->seriesID->{dcmPath,labelPath}}
def getCategorizedDictByFolder(path):
    files = glob.glob(path + "*.dcm")
    dict = {}
    for file in files:
        ds = dcmread(file)
        patientID = ds.PatientID
        studyID = ds.StudyID
        seriesID = str(ds.SeriesNumber)
        labelPath = getLabelPath(file, ds.SOPInstanceUID)
        data = {'dcmPath':file,'labelPath':labelPath}
        if patientID in dict:
            if studyID in dict[patientID]:
                if seriesID in dict[patientID][studyID]:
                    dict[patientID][studyID][seriesID].append(data)
                else:
                    dict[patientID][studyID][seriesID] = [data]
            else:
                dict[patientID][studyID] = {seriesID:[data]}
        else:
            dict[patientID] = {studyID:{seriesID:[data]}}

    # sort dict by InstanceNumber
    for patient in dict:
            for study in dict[patient]:
                for series in dict[patient][study]:
                    dict[patient][study][series].sort(key=lambda x: dcmread(x['dcmPath']).InstanceNumber, reverse=False)

    return dict

def getSeriesIDByName(dict, name):
    for patient in dict:
        for study in dict[patient]:
            for series in dict[patient][study]:
                for i in dict[patient][study][series]:
                    if(name.lower() in dcmread(i['dcmPath']).SeriesDescription.lower()):
                        return series
##################################################################

# 調整圖像色階
def adjust_color_scale(img,Shadow,Highlight):
    if Highlight > 255:
        Highlight = 255
    if Shadow < 0:
        Shadow = 0
    if Shadow >= Highlight:
        Shadow = Highlight - 2
    # 转类型
    img = np.array(img, dtype=int)
    # 计算白场黑场离差
    Diff = Highlight - Shadow
    # 计算系数
    coe = 255.0 / Diff
    rgbDiff = img - Shadow
    rgbDiff = np.maximum(rgbDiff, 0)
    img = rgbDiff * coe
    # 四舍五入到整数
    img = np.around(img, 0)
    # 变为int型
    img = img.astype(np.uint8)
    return img

##################################################################

# 計算兩張圖片的相似度（https://www.twblogs.net/a/5ee8804f38b2869e41a03399）

# 均值哈希算法
def aHash(img):
    # 縮放爲8*8
    img = cv2.resize(img, (8, 8))
    # 轉換爲灰度圖
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # s爲像素和初值爲0，hash_str爲hash值初值爲''
    s = 0
    hash_str = ''
    # 遍歷累加求像素和
    for i in range(8):
        for j in range(8):
            s = s + gray[i, j]
    # 求平均灰度
    avg = s / 64
    # 灰度大於平均值爲1相反爲0生成圖片的hash值
    for i in range(8):
        for j in range(8):
            if gray[i, j] > avg:
                hash_str = hash_str + '1'
            else:
                hash_str = hash_str + '0'
    return hash_str
# 差值感知算法
def dHash(img):
    img = cv2.resize(img, (9, 8))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    hash_str = ''
    # 每行前一個像素大於後一個像素爲1，相反爲0，生成哈希
    for i in range(8):
        for j in range(8):
            if gray[i, j] > gray[i, j + 1]:
                hash_str = hash_str + '1'
            else:
                hash_str = hash_str + '0'
    return hash_str
# 感知哈希算法(pHash)
def pHash(img):
    img = cv2.resize(img, (32, 32))  # , interpolation=cv2.INTER_CUBIC
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # 將灰度圖轉爲浮點型，再進行dct變換
    dct = cv2.dct(np.float32(gray))
    # opencv實現的掩碼操作
    dct_roi = dct[0:8, 0:8]

    hash = []
    avreage = np.mean(dct_roi)
    for i in range(dct_roi.shape[0]):
        for j in range(dct_roi.shape[1]):
            if dct_roi[i, j] > avreage:
                hash.append(1)
            else:
                hash.append(0)
    return hash
# Hash值對比
def cmpHash(hash1, hash2):
    n = 0
    # hash長度不同則返回-1代表傳參出錯
    if len(hash1)!=len(hash2):
        return -1
    # 遍歷判斷
    for i in range(len(hash1)):
        # 不相等則n計數+1，n最終爲相似度
        if hash1[i] != hash2[i]:
            n = n + 1
    return n

##################################################################

# interpolating between two images
import matplotlib.cm as cm
from scipy import interpolate
from scipy.ndimage.measurements import center_of_mass
from scipy.ndimage.interpolation import shift

def inter(images,t):
    # input: 
    # images: list of arrays/frames ordered according to motion
    # t: parameter ranging from 0 to 1 corresponding to first and last frame 
    # returns: interpolated image

    #direction of movement, assumed to be approx. linear 
    a=np.array(center_of_mass(images[0]))
    b=np.array(center_of_mass(images[-1]))

    #find index of two nearest frames
    arr=np.array([center_of_mass(images[i]) for i in range(len(images))])
    v=a+t*(b-a) #convert t into vector 
    idx1 = (np.linalg.norm((arr - v),axis=1)).argmin()
    arr[idx1]=np.array([0,0]) #this is sloppy, should be changed if relevant values are near [0,0]
    idx2 = (np.linalg.norm((arr - v),axis=1)).argmin()

    if idx1>idx2:
        b=np.array(center_of_mass(images[idx1])) #center of mass of nearest contour
        a=np.array(center_of_mass(images[idx2])) #center of mass of second nearest contour
        tstar=np.linalg.norm(v-a)/np.linalg.norm(b-a) #define parameter ranging from 0 to 1 for interpolation between two nearest frames
        im1_shift=shift(images[idx2],(b-a)*tstar) #shift frame 1
        im2_shift=shift(images[idx1],-(b-a)*(1-tstar)) #shift frame 2
        return im1_shift+im2_shift #return average

    if idx1<idx2:
        b=np.array(center_of_mass(images[idx2]))
        a=np.array(center_of_mass(images[idx1]))
        tstar=np.linalg.norm(v-a)/np.linalg.norm(b-a)
        im1_shift=shift(images[idx2],-(b-a)*(1-tstar))
        im2_shift=shift(images[idx1],(b-a)*(tstar))
        return im1_shift+im2_shift


##################################################################

# convert dicom's pixel array to image looks like what we see on standard DICOM viewers
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut, apply_modality_lut

def get_LUT_value_LINEAR_EXACT(data, window, level):
    """
    Adjust according to VOI LUT, window center(level) and width values
    not normalized
    """
    data_min = data.min()
    data_max = data.max()
    data_range = data_max - data_min
    data = np.piecewise(data, 
        [data<=(level-(window)/2),
        data>(level+(window)/2)],
        [data_min, data_max, lambda data: ((data-level+window/2)/window*data_range)+data_min])
    return data

def pixel_process(ds, pixel_array):
    """
    Process the images
    input image info and original pixeal_array
    applying LUTs: Modality LUT -> VOI LUT -> Presentation LUT    
    return processed pixel_array, in 8bit; RGB if color
    """
    
    ## LUTs: Modality LUT -> VOI LUT -> Presentation LUT
    # Modality LUT, Rescale slope, Rescale Intercept
    if 'RescaleSlope' in ds and 'RescaleIntercept' in ds:
        # try applying rescale slope/intercept
        # cannot use INT, because rescale slope could be<1 
        rescale_slope = float(ds.RescaleSlope) # int(ds.RescaleSlope)
        rescale_intercept = float(ds.RescaleIntercept) #  int(ds.RescaleIntercept)
        pixel_array = (pixel_array) * rescale_slope + rescale_intercept
    else:
        # otherwise, try to apply modality 
        pixel_array = apply_modality_lut(pixel_array, ds)


    # personally prefer sigmoid function than window/level
    # personally prefer LINEAR_EXACT than LINEAR (prone to err if small window/level, such as some MR images)
    if 'VOILUTFunction' in ds and ds.VOILUTFunction=='SIGMOID':
        pixel_array = apply_voi_lut(pixel_array, ds)
    elif 'WindowCenter' in ds and 'WindowWidth' in ds:
        window_center = ds.WindowCenter
        window_width = ds.WindowWidth
        # some values may be stored in an array
        if type(window_center)==pydicom.multival.MultiValue:
            window_center = float(window_center[0])
        else:
            window_center = float(window_center)
        if type(window_width)==pydicom.multival.MultiValue:
            window_width = float(window_width[0])
        else:
            window_width = float(window_width)
        pixel_array = get_LUT_value_LINEAR_EXACT(pixel_array, window_width, window_center)
    else:
        # if there is no window center, window width tag, try applying VOI LUT setting
        pixel_array = apply_voi_lut(pixel_array, ds)
        
    # Presentation VOI
    # normalize to 8 bit
    pixel_array = ((pixel_array-pixel_array.min())/(pixel_array.max()-pixel_array.min())) * 255.0
    # if PhotometricInterpretation == "MONOCHROME1", then inverse; eg. xrays
    if 'PhotometricInterpretation' in ds and ds.PhotometricInterpretation == "MONOCHROME1":
        # NOT add minus directly
        pixel_array = np.max(pixel_array) - pixel_array
    
    # conver float -> 8-bit
    return pixel_array.astype('uint8')

##################################################################

# Image shadow and highlight correction
def imageCorrection(
        img,
        shadow_amount_percent, shadow_tone_percent, shadow_radius,
        highlight_amount_percent, highlight_tone_percent, highlight_radius,
        color_percent
):
    """
    Image Shadow / Highlight Correction. The same function as it in Photoshop / GIMP
    :param img: input RGB image numpy array of shape (height, width, 3)
    :param shadow_amount_percent [0.0 ~ 1.0]: Controls (separately for the highlight and shadow values in the image) how much of a correction to make.
    :param shadow_tone_percent [0.0 ~ 1.0]: Controls the range of tones in the shadows or highlights that are modified.
    :param shadow_radius [>0]: Controls the size of the local neighborhood around each pixel
    :param highlight_amount_percent [0.0 ~ 1.0]: Controls (separately for the highlight and shadow values in the image) how much of a correction to make.
    :param highlight_tone_percent [0.0 ~ 1.0]: Controls the range of tones in the shadows or highlights that are modified.
    :param highlight_radius [>0]: Controls the size of the local neighborhood around each pixel
    :param color_percent [-1.0 ~ 1.0]:
    :return:
    """
    shadow_tone = shadow_tone_percent * 255
    highlight_tone = 255 - highlight_tone_percent * 255

    shadow_gain = 1 + shadow_amount_percent * 6
    highlight_gain = 1 + highlight_amount_percent * 6

    # extract RGB channel
    height, width = img.shape[:2]
    img = img.astype(np.float)
    img_R, img_G, img_B = img[..., 2].reshape(-1), img[..., 1].reshape(-1), img[..., 0].reshape(-1)

    # The entire correction process is carried out in YUV space,
    # adjust highlights/shadows in Y space, and adjust colors in UV space
    # convert to Y channel (grey intensity) and UV channel (color)
    img_Y = .3 * img_R + .59 * img_G + .11 * img_B
    img_U = -img_R * .168736 - img_G * .331264 + img_B * .5
    img_V = img_R * .5 - img_G * .418688 - img_B * .081312

    # extract shadow / highlight
    shadow_map = 255 - img_Y * 255 / shadow_tone
    shadow_map[np.where(img_Y >= shadow_tone)] = 0
    highlight_map = 255 - (255 - img_Y) * 255 / (255 - highlight_tone)
    highlight_map[np.where(img_Y <= highlight_tone)] = 0

    # // Gaussian blur on tone map, for smoother transition
    if shadow_amount_percent * shadow_radius > 0:
        # shadow_map = cv2.GaussianBlur(shadow_map.reshape(height, width), ksize=(shadow_radius, shadow_radius), sigmaX=0).reshape(-1)
        shadow_map = cv2.blur(shadow_map.reshape(height, width), ksize=(shadow_radius, shadow_radius)).reshape(-1)

    if highlight_amount_percent * highlight_radius > 0:
        # highlight_map = cv2.GaussianBlur(highlight_map.reshape(height, width), ksize=(highlight_radius, highlight_radius), sigmaX=0).reshape(-1)
        highlight_map = cv2.blur(highlight_map.reshape(height, width), ksize=(highlight_radius, highlight_radius)).reshape(-1)

    # Tone LUT
    t = np.arange(256)
    LUT_shadow = (1 - np.power(1 - t * (1 / 255), shadow_gain)) * 255
    LUT_shadow = np.maximum(0, np.minimum(255, np.int_(LUT_shadow + .5)))
    LUT_highlight = np.power(t * (1 / 255), highlight_gain) * 255
    LUT_highlight = np.maximum(0, np.minimum(255, np.int_(LUT_highlight + .5)))

    # adjust tone
    shadow_map = shadow_map * (1 / 255)
    highlight_map = highlight_map * (1 / 255)

    iH = (1 - shadow_map) * img_Y + shadow_map * LUT_shadow[np.int_(img_Y)]
    iH = (1 - highlight_map) * iH + highlight_map * LUT_highlight[np.int_(iH)]
    img_Y = iH

    # adjust color
    if color_percent != 0:
        # color LUT
        if color_percent > 0:
            LUT = (1 - np.sqrt(np.arange(32768)) * (1 / 128)) * color_percent + 1
        else:
            LUT = np.sqrt(np.arange(32768)) * (1 / 128) * color_percent + 1

        # adjust color saturation adaptively according to highlights/shadows
        color_gain = LUT[np.int_(img_U ** 2 + img_V ** 2 + .5)]
        w = 1 - np.minimum(2 - (shadow_map + highlight_map), 1)
        img_U = w * img_U + (1 - w) * img_U * color_gain
        img_V = w * img_V + (1 - w) * img_V * color_gain

    # re convert to RGB channel
    output_R = np.int_(img_Y + 1.402 * img_V + .5)
    output_G = np.int_(img_Y - .34414 * img_U - .71414 * img_V + .5)
    output_B = np.int_(img_Y + 1.772 * img_U + .5)

    output = np.row_stack([output_B, output_G, output_R]).T.reshape(height, width, 3)
    output = np.minimum(output, 255).astype(np.uint8)
    return output

##################################################################

# Rescale array
def rescaleFrom0to255(arr):
    min_val = arr.min()
    max_val = arr.max()

    # Rescale the array
    arr_rescaled = (arr - min_val) * (255 - 0) / (max_val - min_val) + 0
    return arr_rescaled.astype(np.uint8)

# Rescale array
def rescaleFrom0to1(arr):
    min_val = arr.min()
    max_val = arr.max()

    # Rescale the array
    arr_rescaled = (arr - min_val) * (1 - 0) / (max_val - min_val) + 0
    return arr_rescaled

##################################################################

# applu CT window
def apply_ct_window(img, window):
    # window = (window width, window level)
    R = (img-window[1]+0.5*window[0])/window[0]
    R[R<0] = 0
    R[R>1] = 1
    return R

# enlarge image layout
def enlargeImageLayout(input,outputSize):
    size = input.shape
    output = np.zeros((outputSize))
    startX = outputSize[0]//2 - size[0]//2
    startY = outputSize[1]//2 - size[1]//2
    output[startX:startX+size[0],startY:startY+size[1]] = input
    return output

In [ ]:
#superimpose original image and label
path = "20221005/001/001-MR/"
images_path = glob.glob(path + '*_0.png')
labels_path = glob.glob(path + '*_anno.png')
images = []
for i in range(len(images_path)):
    image = Image.open(images_path[i])
    label = Image.open(labels_path[i])
    image.paste(label,(0,0),mask=label)
    images.append(image)
    
#show superimposed images with pyplot
showImagesWithPyPlot(images)

In [ ]:
#draw lines with pillow and add images to list
path = "20221005/002/002-MR/"
images_path = glob.glob(path + '*_0.png')
labels_path = glob.glob(path + '*_anno.png')
jsonFiles = glob.glob(path + "*.json")

labelImages = []
for i in jsonFiles:
    labelImages.append(getLabeledImageWithJson(i))

#show filledLabels with pyplot
print("Quantity: " + str(len(labelImages)))
showImagesWithPyPlot(labelImages)
    
#superimpose filledLabelImages to original mri images and show
images=[]
for i in range(len(labelImages)):
    image = Image.open(images_path[i])
    labelImage = labelImages[i]
#     alpha = Image.new("L", labelImage.size, 100)
#     labelImage.putalpha(alpha)
    image.paste(labelImage,(0,0),mask=labelImage)
    images.append(image)
showImagesWithPyPlot(images)

In [ ]:
# categorize dicom files with patientID, studyID and seriesID using dicomdir
# dict: {patientID->studyID->seriesID->{dcmPath,label}}
# find images file name by SeriesNumber to be drawn
from pydicom.fileset import FileSet
dicomdir = dcmread("002/dicomdir")
fs = FileSet(dicomdir)
# print(fs)

dict = {}
patientIDs = fs.find_values("PatientID")
for patientID in patientIDs:
    instances = fs.find(PatientID=patientID)
    for ds in instances:
        patientID = ds.PatientID
        studyID = ds.StudyID
        seriesID = ds.SeriesNumber
        fileName = ds.ReferencedFileID
        data = {'dcmPath':fileName,'labelPath':'NA'}
        if patientID in dict:
            if studyID in dict[patientID]:
                if seriesID in dict[patientID][studyID]:
                    dict[patientID][studyID][seriesID].append(data)
                else:
                    dict[patientID][studyID][seriesID] = [data]
            else:
                dict[patientID][studyID] = {seriesID:[data]}
        else:
            dict[patientID] = {studyID:{seriesID:[data]}}

#draw images
drawCategorizedImages('002/', dict, fontsize=15, fontfamily='DejaVu Sans')

In [13]:
# 按照series對MRI與PET做資料夾分類，并且畫出MRI對應的標記圖像
patientFolder = '004'
mri_dict = getCategorizedDictByFolder('./20221005/'+ patientFolder+ '/'+ patientFolder+ '-MR/')
pet_dict = getCategorizedDictByFolder('./20221005/'+ patientFolder+ '/'+ patientFolder+ '-PT/')

for patient in mri_dict:
    for study in mri_dict[patient]:
        for series in mri_dict[patient][study]:
            seriesDescripiton = dcmread(mri_dict[patient][study][series][0]['dcmPath']).SeriesDescription
            seriesDescripiton = seriesDescripiton.replace('/',' ')
            path = 'categorized/'+ patientFolder+ '/MRI/'+ series + ' (' + seriesDescripiton + ')'
            labelPath = 'categorized/'+ patientFolder+ '/MRI/'+ series + ' (' + seriesDescripiton + ')' + '/label/'
            if not os.path.exists(path):
                os.makedirs(path)
            if not os.path.exists(labelPath):
                os.makedirs(labelPath)
            for c,v in enumerate(mri_dict[patient][study][series]):
                shutil.copy(v['dcmPath'],path+ '/'+ str(c+1).zfill(3)+ '.dcm')
                width = dcmread(v['dcmPath']).pixel_array.shape[0]
                height = dcmread(v['dcmPath']).pixel_array.shape[1]
                labelImg = Image.new('RGBA', (width, height))
                if(v['labelPath']!='NA'):                    
                    labelImg = getLabeledImageWithJson(v['labelPath'])
                labelImg.save(labelPath + str(c+1).zfill(3) + '.png')                  
                
for patient in pet_dict:
    for study in pet_dict[patient]:
        for series in pet_dict[patient][study]:
            seriesDescripiton = dcmread(pet_dict[patient][study][series][0]['dcmPath']).SeriesDescription
            seriesDescripiton = seriesDescripiton.replace('/',' ')
            path = 'categorized/'+ patientFolder+ '/PET/'+ series + ' (' + seriesDescripiton + ')'
            if not os.path.exists(path):
                os.makedirs(path)
            for c,v in enumerate(pet_dict[patient][study][series]):
                shutil.copy(v['dcmPath'],path+ '/'+ str(c+1).zfill(3)+ '.dcm')

In [191]:
# convert MRI dicom to png 
import dicom2jpg
mriDicomFolder = 'categorized/005/MRI/6 (t2_tse_dark-fluid_fs_tra+C)/'
dicom2jpg.dicom2png(mriDicomFolder, target_root=mriDicomFolder)

True

In [311]:
# convert PET dicom to png 
import dicom2jpg
petDicomFolder = 'categorized/005/PET/1353 (FDG WBS PET CT Axial)/'
dicom2jpg.dicom2png(petDicomFolder, target_root=petDicomFolder)

True

In [320]:
# rename files (因轉換後檔案名稱在排序上有問題，需要重新命名)
startNum = 7
renameFolder = 'categorized/001/PET/1353 (FDG WBS PET CT Axial)/png/'
files = glob.glob(renameFolder + "*.png")
files.sort(key=lambda x: len(x), reverse=False)
for c,v in enumerate(files):
    os.rename(v, renameFolder + str(c+startNum).zfill(3) + '.png')

In [ ]:
# try to remove hand from PET image
ds= io.imread('C:/Users/Justin/Desktop/MRI/pet/gray/003/040.png')
cols = ds.shape[0]
rows = ds.shape[1]
p = 0
for i in range(cols):
    for j in range(rows//2,rows):
        if(ds[j][i]==0):
            p = j
            break
    for k in range(p,rows):
        ds[k][i]=0

for i in range(cols):
    for j in range(rows//2,0,-1):
        if(ds[j][i]==0):
            p = j
            break
    for k in range(p,0,-1):
        ds[k][i]=0

plt.imshow(ds,cmap='gray')

In [ ]:
# Find the most similar image
from image_similarity_measures.quality_metrics import rmse,ssim,fsim,sre
flairImg = cv2.imread('categorized/005/MRI/6 (t2_tse_dark-fluid_fs_tra+C)/png/025.png')
flairImg = cv2.resize(flairImg, (512, 512))
petPaths = glob.glob('pet/005/*.png')

similarityDict = []
print('No,SSIM,FSIM,RMSE,SRE,PHash')
for i in range(28,40):
    img2 = cv2.imread(petPaths[i])
    _ssim = ssim(flairImg,img2)
    _fsim = fsim(flairImg,img2)
    _rmse = rmse(flairImg,img2)
    _sre = sre(flairImg,img2)
    _pHash = cmpHash(pHash(flairImg),pHash(img2))
    print(str(i+1),end=',')
    print(str(_ssim),end=',')
    print(str(_fsim),end=',')
    print(str(_rmse),end=',')
    print(str(_sre),end=',')
    print(str(_pHash))
    similarityDict.append({'No':i+1,'SSIM':_ssim,'FSIM':_fsim,'RMSE':_rmse,'SRE':_sre,'PHash':_pHash})
similarityDict.sort(key=lambda x:x['SSIM'],reverse=True)
print(similarityDict[0]['No'])


ImageRegistration:

In [327]:
mriPath = 'categorized/002/MRI/6 (t2_tse_dark-fluid_fs_tra+C)/'
ctPath = 'categorized/002/PET/3 (WB Standard)/'
patientNum = '002'
startImgNum = 12

In [258]:
# 挑選MRI内插後與CT位置對應的圖像
# 14*(n-1)+n
mriSliceNum = 25    #mri對應號碼
ctSliceNum = 35    #ct對應號碼
start = 14*(mriSliceNum-1)+mriSliceNum
src_path = glob.glob('C:/Users/Justin/Desktop/high_order_slices_interpolation_for-medical_images/interpolated/mri/'+patientNum+'/*.png')
dst_path = 'corresponded/'+patientNum+'/label/'
if not os.path.exists(dst_path):
    os.makedirs(dst_path)
n = 0
m = ctSliceNum
print(start)
for i in range(start,0,-1):
    if(n%28==0):
        shutil.copy(src_path[i],dst_path+str(m).zfill(3)+'.png')
        n=0
        m-=1
    n+=2
m = ctSliceNum
for i in range(start,len(src_path)):
    if(n%28==0):
        shutil.copy(src_path[i],dst_path+str(m).zfill(3)+'.png')
        n=0
        m+=1
    n+=2

361


In [ ]:
# 處理内插后的label（<0.5設爲0，>=0.5設爲1）
patientNum = '005'
startImgNum = 10
labelFolder = glob.glob('corresponded/'+patientNum+'/label/shaped/*.png')
savePath = 'corresponded/'+patientNum+'/label/shaped/'
if not os.path.exists(savePath):
    os.makedirs(savePath)
for i in range(len(labelFolder)):
    label = io.imread(labelFolder[i])
    label[label<127] = 0
    label[label>=127] = 255
    io.imsave(savePath+str(i+startImgNum).zfill(3)+'.png',label)

In [ ]:
# 計算MRI與PET之間的voxel物理大小 與 兩者之間的縮放比例
mri = dcmread(mriPath + '001.dcm')
ct = dcmread(ctPath + '001.dcm')
mriPixelSpacing = mri.PixelSpacing
ctPixelSpacing = ct.PixelSpacing
print(mri.pixel_array.shape)
print('MRI: ' + str(mriPixelSpacing))
print(ct.pixel_array.shape)
print('CT: ' + str(ctPixelSpacing))
print('Ratio: ' + str(mriPixelSpacing[0]/ctPixelSpacing[0]))

In [ ]:
# rotate and rescale mr images
from skimage.transform import rotate,rescale,resize
from scipy.ndimage import shift
mirFolder = mriPath
ctFolder = ctPath
correspondedMRIPaths = glob.glob('corresponded/'+patientNum+'/label/*.png')
saveFolder = 'corresponded/'+patientNum+'/label/shaped/'
if not os.path.exists(saveFolder):
    os.makedirs(saveFolder)

mri = dcmread(mirFolder + '001.dcm')
mri2 = dcmread(mirFolder + '002.dcm')
ct = dcmread(ctFolder + '001.dcm')
mriPixelSpacing = mri.PixelSpacing
ctPixelSpacing = ct.PixelSpacing
imgSize = ct.pixel_array.shape[0]   
rotateVal = 4.7
shiftX = -11
shiftY = 0
mmRescaleVal = mriPixelSpacing[0]/ctPixelSpacing[0]  #物理比例
deltaXVal = (mri2.ImagePositionPatient[0] - mri.ImagePositionPatient[0]) / mri.PixelSpacing[0]  #mm轉換成像素單位
deltaYVal = (mri2.ImagePositionPatient[1] - mri.ImagePositionPatient[1]) / mri.PixelSpacing[0]  #mm轉換成像素單位
deltaX = 0
deltaY = 0
print(mmRescaleVal)
for i in range(0,len(correspondedMRIPaths)):
    img = io.imread(correspondedMRIPaths[i])
    img = rescale(img,mmRescaleVal)
    img = rotate(img,rotateVal)
    img = enlargeImageLayout(img,(512,512))
    img = shift(img,(shiftY,shiftX))
    io.imsave(saveFolder+str(i+startImgNum).zfill(3)+'.png', img)

In [ ]:
# 對應好的MRI與標記檔進行縮放，對應到50%PET的大小
from skimage.transform import rescale
from skimage.color import rgb2gray
patientNum = '005'
startImgNum = 10
rescaleVal = 1.6
outputSize = 320
correspondedMRIPaths = glob.glob('corresponded/'+patientNum+'/shaped/*.png')
correspondedLabelPaths = glob.glob('corresponded/'+patientNum+'/label/shaped/*.png')
petPaths = glob.glob('corresponded/'+patientNum+'/pet/*.png')
saveMriPath = 'corresponded/'+patientNum+'/shaped/'
saveLabelPath = 'corresponded/'+patientNum+'/label/shaped/'
savePetPath = 'corresponded/'+patientNum+'/pet/'
saveGrayPetPath = 'corresponded/'+patientNum+'/pet/gray/'
if not os.path.exists(saveGrayPetPath):
    os.makedirs(saveGrayPetPath)
for i in range(len(correspondedMRIPaths)):
    mri = io.imread(correspondedMRIPaths[i])
    label = io.imread(correspondedLabelPaths[i])
    pet = io.imread(petPaths[i])
    mri = rescale(mri,rescaleVal)
    label = rescale(label,rescaleVal)
    mriStartPos = (mri.shape[0] - outputSize) // 2
    petStartPos = (pet.shape[0] - outputSize) // 2
    mri = mri[mriStartPos:mriStartPos+outputSize,mriStartPos:mriStartPos+outputSize]
    label = label[mriStartPos:mriStartPos+outputSize,mriStartPos:mriStartPos+outputSize]
    pet = pet[petStartPos:petStartPos+outputSize,petStartPos:petStartPos+outputSize,:]
    grayPet = rgb2gray(pet)
    io.imsave(saveMriPath+str(i+startImgNum).zfill(3)+'.png',mri)
    io.imsave(saveLabelPath+str(i+startImgNum).zfill(3)+'.png',label)
    io.imsave(savePetPath+str(i+startImgNum).zfill(3)+'.png',pet)
    io.imsave(saveGrayPetPath+str(i+startImgNum).zfill(3)+'.png',grayPet)

In [ ]:
# convert color pet to gray pet and save
patientNum = '005'
startImgNum = 13
petPaths = glob.glob('corresponded/'+patientNum+'/pet/*.png')
saveGrayPetPath = 'corresponded/'+patientNum+'/pet/gray/'
if not os.path.exists(saveGrayPetPath):
    os.makedirs(saveGrayPetPath)
for i in range(len(petPaths)):
    pet = io.imread(petPaths[i])
    grayPet = rgb2gray(pet)
    io.imsave(saveGrayPetPath+str(i+startImgNum).zfill(3)+'.png',grayPet)

In [ ]:
#superimpose label to mri and pet then save image
patientNum = '004'
startImgNum = 10
correspondedMRIPaths = glob.glob('corresponded/'+patientNum+'/shaped/*.png')
correspondedLabelPaths = glob.glob('corresponded/'+patientNum+'/label/shaped/contour/*.png')
petPaths = glob.glob('corresponded/'+patientNum+'/pet/*.png')
cols = 2
rows = len(petPaths)
fig = plt.figure(figsize=(20,10))
for n_slice in range(rows):
    mri = Image.open(correspondedMRIPaths[n_slice])
    label = Image.open(correspondedLabelPaths[n_slice])
    pet = Image.open(petPaths[n_slice])
    mri.paste(label,(0,0),mask=label)
    pet.paste(label,(0,0),mask=label)
    ax=fig.add_subplot(1,cols,1)
    ax.set_title("MRI",fontsize=22)
    plt.imshow(mri,cmap='gray')
    plt.axis('off')
    ax=fig.add_subplot(1,cols,2)
    ax.set_title("PET",fontsize=22)
    plt.imshow(pet,cmap='gray')
    plt.axis('off')
    plt.savefig(str(n_slice+1).zfill(3) + '.jpg')

In [ ]:
#draw comparison of FLAIR and shaped PET
patientNum = '003'
mriFolder = glob.glob('corresponded/'+patientNum+'/shaped/*.png')
petFolder = glob.glob('corresponded/'+patientNum+'/pet/*.png')
labelFolder = glob.glob('corresponded/'+patientNum+'/label/shaped/*.png')
cols = 3
rows = len(mriFolder)
fig = plt.figure(figsize=(6, 100))
for n_slice in range(rows):
    ax=fig.add_subplot(rows,cols,(n_slice*cols)+1)
    img1 = io.imread(mriFolder[n_slice])
    plt.imshow(img1,cmap='gray')
    plt.axis('off')
    ax=fig.add_subplot(rows,cols,(n_slice*cols)+2)
    img2 = io.imread(petFolder[n_slice])
    plt.imshow(img2,cmap='gray')
    plt.axis('off')
    ax=fig.add_subplot(rows,cols,(n_slice*cols)+3)
    img3 = io.imread(labelFolder[n_slice])
    plt.imshow(img3,cmap='gray')
    plt.axis('off')

---------------------------------------------------------------------

In [ ]:
# print deltaX, deltaY and deltaZ of slices of dicom
dcmFolder = glob.glob('categorized/004/MRI/601 (T2W_FLAIR_FS+C(Gadovist))/*dcm')
dss = [dcmread(i) for i in dcmFolder]
preX = 0
preY = 0
preZ = 0
for i in dss:
    # print('SliceLocation: ' + str(i.ImagePositionPatient))
    pos = i.ImagePositionPatient
    curX = pos[0]
    curY = pos[1]
    curZ = pos[2]
    print(str(curX - preX)+', '+str(curY - preY)+ ', ' + str(curZ - preZ))
    preX = curX
    preY = curY
    preZ = curZ

In [30]:
#Prepare dataset for input->(SIZE,SIZE,SLICE,CHANNEL)
SIZE = 240
DEPTH = 48
CHANNEL = 3
patientNum = '001'
saveFolder = 'mri_training_dataset/'
t1Folder = glob.glob('categorized/004/MRI/801 (T1W_FFE_FS+CM(Gadovist))/*.dcm')
t2Folder =  glob.glob('categorized/004/MRI/401 (T2W_FS +CM(Gadovist))/*.dcm')
flairFolder =  glob.glob('categorized/004/MRI/601 (T2W_FLAIR_FS+C(Gadovist))/*.dcm')
labelFolder = glob.glob('categorized/004/MRI/label/*.png')

t1 = np.zeros((SIZE,SIZE,DEPTH))
t2 = np.zeros((SIZE,SIZE,DEPTH))
flair = np.zeros((SIZE,SIZE,DEPTH))
seg = np.zeros((SIZE,SIZE,DEPTH))

for c, i in enumerate(t1Folder):
    t1[:,:,c] = cv2.resize(rescaleFrom0to255(dcmread(i).pixel_array), (SIZE,SIZE))
for c, i in enumerate(t2Folder):
    t2[:,:,c] = cv2.resize(rescaleFrom0to255(dcmread(i).pixel_array), (SIZE,SIZE))
for c, i in enumerate(flairFolder):
    flair[:,:,c] = cv2.resize(rescaleFrom0to255(dcmread(i).pixel_array), (SIZE,SIZE))
for c, i in enumerate(labelFolder):
    seg[:,:,c] = cv2.resize(io.imread(i)[:,:,2], (SIZE,SIZE))
seg[seg!=0] = 1
nib.save(nib.Nifti1Image(t1, affine=np.eye(4)), saveFolder+'mri_'+patientNum+'_t1.nii')
nib.save(nib.Nifti1Image(t2, affine=np.eye(4)), saveFolder+'mri_'+patientNum+'_t2.nii')
nib.save(nib.Nifti1Image(flair, affine=np.eye(4)), saveFolder+'mri_'+patientNum+'_flair.nii')
nib.save(nib.Nifti1Image(seg, affine=np.eye(4)), saveFolder+'mri_'+patientNum+'_seg.nii')

In [ ]:
#validate nii mri dataset
seg = nib.load('mri_training_dataset/mri_004_seg.nii').get_fdata()
flair = nib.load('mri_training_dataset/mri_004_flair.nii').get_fdata()
t1 = nib.load('mri_training_dataset/mri_004_t1.nii').get_fdata()
t2 = nib.load('mri_training_dataset/mri_004_t2.nii').get_fdata()
cols=4
rows=48
fig = plt.figure(figsize=(8, 100))
for n_slice in range(rows):
    ax=fig.add_subplot(rows,cols,(n_slice*cols)+1)
    ax.title.set_text('flairs')
    plt.imshow(flair[:,:,n_slice], cmap='gray')
    plt.axis('off')
    ax=fig.add_subplot(rows,cols,(n_slice*cols)+2)
    ax.title.set_text('t1')
    plt.imshow(t1[:,:,n_slice], cmap='gray')
    plt.axis('off')
    ax=fig.add_subplot(rows,cols,(n_slice*cols)+3)
    ax.title.set_text('t2')
    plt.imshow(t2[:,:,n_slice], cmap='gray')
    plt.axis('off')
    ax=fig.add_subplot(rows,cols,(n_slice*cols)+4)
    ax.title.set_text('Label')
    plt.imshow(seg[:,:,n_slice])
    plt.axis('off')

In [ ]:
### 嘗試用標記檔配對dcm檔
path = "20221005/002/002-MR"
files = glob.glob(path + "/*.dcm")
pngs = glob.glob(path + "/*0.png")
labes = glob.glob(path + "/*anno.png")
cols = 3
rows = len(pngs)
fig = plt.figure(figsize=(rows/6,rows*2))
for i in range(len(pngs)):
    sopID = str(pngs[i].split(path)[1][1:-6]) #去除資料夾字段及後綴，保留UID
    for j in files:
        f = dcmread(j)
        if(str(f.SOPInstanceUID) == sopID):
            ax1 = fig.add_subplot(rows,cols,(i*cols)+1)
            ax1.title.set_text('dcm/series: ' + str(f.SeriesNumber))
            plt.imshow(f.pixel_array)
            plt.axis('off')
            ax2 = fig.add_subplot(rows,cols,(i*cols)+2)
            ax2.title.set_text('png')
            plt.imshow(Image.open(pngs[i]))
            plt.axis('off')
            ax3 = fig.add_subplot(rows,cols,(i*cols)+3)
            ax3.title.set_text('label')
            plt.imshow(Image.open(labes[i]))
            plt.axis('off')

In [ ]:
def save_nii_to_png(niifile):    
    img = nib.load(niifile)
    fdata = img.get_fdata()
    (x,y,z) = fdata.shape
    for i in range(z):
        slice_pic = fdata[:,:,i]
        plt.imsave('nii_to_png/img' + str(i+1) + '.png', slice_pic, cmap='gray')

TRAIN_DATASET_PATH = ''
test_image = nib.load(TRAIN_DATASET_PATH + 'BraTS20_Training_001_seg.nii')
print(test_image)
test_image = test_image.get_fdata()
plt.figure(figsize=(20,20))

for i in range(test_image.shape[2]):
    plt.subplot(11,15,i+1)
    plt.imshow(test_image[:,:,i])
    plt.axis('off')

if(not os.path.exists('nii_to_png')):
    os.mkdir('nii_to_png')
save_nii_to_png(TRAIN_DATASET_PATH + 'BraTS20_Training_001_seg.nii')

In [ ]:
#draw comparison of original and interpolated images
cols = 3
rows = 47

fig = plt.figure(figsize=(6,100))
for n_slice in range(rows-1):
    ax=fig.add_subplot(rows,cols,(n_slice*cols)+1)
    # ax.title.set_text(str(n_slice*2))
    ax.title.set_text('interpolated')
    plt.imshow(flair_interpolated[:,:,(n_slice*4)+1],cmap='gray')
    plt.axis('off')
    ax=fig.add_subplot(rows,cols,(n_slice*cols)+2)
    ax.title.set_text('interpolated')
    plt.imshow(flair_interpolated[:,:,(n_slice*4)+2],cmap='gray')
    plt.axis('off')
    ax=fig.add_subplot(rows,cols,(n_slice*cols)+3)
    # ax.title.set_text(str((n_slice*2)+2))
    ax.title.set_text('interpolated')
    plt.imshow(flair_interpolated[:,:,(n_slice*4)+3],cmap='gray')
    plt.axis('off')

In [ ]:
#calculate images similarity
from image_similarity_measures.quality_metrics import ssim

slices = 47
total_ssim = 0
for i in range(slices-1):
    a = (interpolate_output[:,:,(i*4)+1]/np.max(interpolate_output[:,:,(i*4)+1]))*255
    b = (flair_interpolated[:,:,(i*4)+1])*255
    total_ssim += ssim(a,b)
    a = (interpolate_output[:,:,(i*4)+2]/np.max(interpolate_output[:,:,(i*4)+2]))*255
    b = (flair_interpolated[:,:,(i*4)+2])*255
    total_ssim += ssim(a,b)
    a = (interpolate_output[:,:,(i*4)+3]/np.max(interpolate_output[:,:,(i*4)+3]))*255
    b = (flair_interpolated[:,:,(i*4)+3])*255
    total_ssim += ssim(a,b)
print(total_ssim)
print("total_similarity: "+  str(total_ssim/((slices-1)*3)))
# fig = plt.figure(figsize=(4,2))
# ax=fig.add_subplot(1,2,1)
# plt.imshow(a,cmap='gray')
# plt.axis('off')
# ax=fig.add_subplot(1,2,2)
# plt.imshow(b,cmap='gray')
# plt.axis('off')

In [ ]:
# compare original mri, our method and registration method
from skimage.transform import rotate
mri = glob.glob('categorized/005/MRI/6 (t2_tse_dark-fluid_fs_tra+C)/*.dcm')
registered = nib.load('../registered/001_6dof.nii.gz').get_fdata()
shaped = glob.glob('./pet/005/corresponded/shaped/*.png')
rows = registered.shape[2]
cols = 3
print(np.unique(registered[:,:,25]))
print(registered[:,:,25].shape)
io.imshow(registered[:,:,25],cmap='gray')
fig = plt.figure(figsize=(6,100))
for n_slice in range(rows-1):
    ax=fig.add_subplot(rows,cols,(n_slice*cols)+1)
    plt.imshow(dcmread(mri[n_slice]).pixel_array,cmap='gray')
    plt.axis('off')
    ax=fig.add_subplot(rows,cols,(n_slice*cols)+2)
    plt.imshow(io.imread(shaped[n_slice]),cmap='gray')
    plt.axis('off')
    ax=fig.add_subplot(rows,cols,(n_slice*cols)+3)
    plt.imshow(rotate(registered[:,:,rows-1-n_slice][::-1,:],-90),cmap='gray')
    plt.axis('off')

In [ ]:
#save registered nib to png
from skimage.transform import rotate
ds = dcmread('./categorized/005/PET/3 (WB Standard)/001.dcm')
folder = './registered/005_6dof.nii.gz'
registered = nib.load(folder).get_fdata()
sn = registered.shape[2]
saveFolder = folder[:-7]
if not os.path.exists(saveFolder):
    os.mkdir(saveFolder)
for i in range(registered.shape[2]):
    save_img = apply_ct_window(rotate(registered[:,:,i][::-1,:],-90),[ds.WindowWidth,ds.WindowCenter])
    io.imsave(saveFolder + '/' + str(sn).zfill(3) + '.png',save_img)
    sn-=1

In [ ]:
#save petct and label to nii for training
from skimage.transform import resize
patientNum = '005'
pets = glob.glob('./corresponded/' + patientNum + '/pet/gray/*.png')
labels = glob.glob('./corresponded/' + patientNum + '/label/shaped/*.png')
imgSize = 320
slicesNum = 48
output = np.zeros((imgSize,imgSize,slicesNum))
seg = np.zeros((imgSize,imgSize,slicesNum))
for i in range(slicesNum):
    if(i>=len(pets)):
        output[:,:,i] = np.zeros((imgSize,imgSize))
        seg[:,:,i] = np.zeros((imgSize,imgSize))
    else:
        img = io.imread(pets[i])
        label = io.imread(labels[i])[:,:]
        print(np.unique(label))
        if(img.shape[0]>256):
            img = rescaleFrom0to255(resize(img,(imgSize,imgSize)))
        if(label.shape[0]>256):
            label = rescaleFrom0to255(resize(label,(imgSize,imgSize)))
        output[:,:,i] = img
        seg[:,:,i] = label
seg[seg!=0] = 1
nib.save(nib.Nifti1Image(output, affine=np.eye(4)), 'pet_training_dataset/' + patientNum + '_pet.nii.gz')
nib.save(nib.Nifti1Image(seg, affine=np.eye(4)), 'pet_training_dataset/' + patientNum + '_pet_seg.nii.gz')

In [ ]:
# validate saved pet nib (compare with original mri)
patientNum = '001'
mri = glob.glob('./corresponded/'+patientNum+'/shaped/*.png')
pet = nib.load('./pet_training_dataset/'+patientNum+'_pet.nii.gz').get_fdata()
seg = nib.load('./pet_training_dataset/'+patientNum+'_pet_seg.nii.gz').get_fdata()
print(pet.shape)
print(seg.shape)
print(np.unique(seg))
cols = 3
rows = 48
fig = plt.figure(figsize=(6,100))
for n_slice in range(rows):
    ax=fig.add_subplot(rows,cols,(n_slice*cols)+1)
    plt.imshow(io.imread(mri[n_slice]),cmap='gray')
    plt.axis('off')
    ax=fig.add_subplot(rows,cols,(n_slice*cols)+2)
    plt.imshow(pet[:,:,n_slice],cmap='gray')
    plt.axis('off')
    ax=fig.add_subplot(rows,cols,(n_slice*cols)+3)
    plt.imshow(seg[:,:,n_slice])
    plt.axis('off')

In [ ]:
#superimpose MRI and PET with label and save
mriFolder = "./categorized/004/MRI/601 (T2W_FLAIR_FS+C(Gadovist))/png/"
petFolder = "../registered/png/"
labelFolder = "./categorized/004/MRI/label/"
mri_path = glob.glob(mriFolder + '*.png')
pet_path = glob.glob(petFolder + "*.png")
labels_path = glob.glob(labelFolder + '*.png')
mriLabeledImages = []
petLabeledImages = []
sliceSize = len(mri_path)
for i in range(sliceSize):
    image = Image.open(mri_path[i])
    rgbImage = Image.new("RGB", image.size)
    rgbImage.paste(image)
    label = Image.open(labels_path[i])
    rgbImage.paste(label,(0,0),mask=label)
    mriLabeledImages.append(rgbImage)
    image = Image.open(pet_path[sliceSize-i-1])
    rgbImage = Image.new("RGB", image.size)
    rgbImage.paste(image)
    rgbImage.paste(label,(0,0),mask=label)
    petLabeledImages.append(rgbImage)

rows = 1
cols = 2
fig = plt.figure(figsize=(20,10))
for n_slice in range(sliceSize):
    ax=fig.add_subplot(rows,cols,1)
    ax.set_title("MRI",fontsize=22)
    plt.imshow(mriLabeledImages[n_slice],cmap='gray')
    plt.axis('off')
    ax=fig.add_subplot(rows,cols,2)
    ax.set_title("PET",fontsize=22)
    plt.imshow(petLabeledImages[n_slice],cmap='gray')
    plt.axis('off')
    plt.savefig(str(n_slice+1).zfill(3) + '.jpg')